## Step 4 - Sentiment Analysis labels

#### In this notebook we:
a. read the data_folder/tweets_table_all.csv - this is a table with ~1M tweets, containing tweets of KOL ad tweets of brexit from 2014-2021. It is a table **without sentiment**

b. Import the fine-tuned sentiment model - from the location: Brexit_sentiment_model

c. We use the sentiment model to give sentiment-prediction-labels to the ~1M tweets (from a)
We save the labeled data frame in the following location:  data_folder/tweets_table_all_with_sentimet.csv 
We will use that table in step 5

In [1]:
%matplotlib inline
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import gensim, logging, pandas as pd
import numpy as np, matplotlib.pyplot as plt, os
import pandas as pd
import json, zipfile
import torch
import seaborn as sns
import warnings

from transformers import pipeline
from scipy.special import softmax
from transformers import AutoTokenizer, AutoConfig
from tqdm import tqdm


device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu") 
print(f"Will be training on {device}")


Will be training on cpu


### 4.a Reading tweets_table_all (~1M tweets)
read the data_folder/tweets_table_all.csv - this is a table with ~1M tweets, containing tweets of KOL ad tweets of brexit from 2014-2021. It is a table without sentiment

In [11]:
tweets_table_all = pd.read_csv("data_folder/tweets_table_all.csv")
tweets_table_all = tweets_table_all.drop(labels = ["Unnamed: 0"], axis = 1)

C:\Users\roymad\AppData\Local\Temp\ipykernel_39852\4151046788.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  tweets_table_all = pd.read_csv("data_folder/tweets_table_all.csv")


### 4.b Using the sentiment model that we fine-tuned on Step 3 to make sentiment analysis predictions
Import the fine-tuned sentiment model - from the location: Brexit_sentiment_model

In [5]:
import os.path
name_dir_to_save_model = "Brexit_sentiment_model" #The name of the foler containing the fine-tuned sentiment model

# try:
#     os.makedirs(name_dir_to_save_model)
#     print("Creating directory", name_dir_to_save_model, " to save the sentiment analysis model")
# except:
#     print("The dir: ", name_dir_to_save_model, "already exists")
# ### saving the model we re-trained

### if you wosh to read the saved model:
sentiment_task_example = pipeline("sentiment-analysis",
model = "cardiffnlp/twitter-roberta-base-sentiment-latest", 
tokenizer = "cardiffnlp/twitter-roberta-base-sentiment-latest")

model_loaded = sentiment_task_example.model.from_pretrained(name_dir_to_save_model)

sentiment_task_model_loaded = pipeline("sentiment-analysis",
model = model_loaded, 
tokenizer = "cardiffnlp/twitter-roberta-base-sentiment-latest")


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Defining function for getting the sentiment from all the tweets

+ if you define yourself the "sentiment task" object then you can save a lot of time!
 sentiment_task = pipeline("sentiment-analysis", model = model, #this is the model we re-trained
            tokenizer = "cardiffnlp/twitter-roberta-base-sentiment-latest")

+ x = the text to classify

In [6]:
def Sentiment_predictions_huggin_face(x, sentiment_task = None, re_trained_model = None, 
tokenizer = "cardiffnlp/twitter-roberta-base-sentiment-latest"):

    if sentiment_task is None:
        sentiment_task = pipeline("sentiment-analysis",
            model = re_trained_model, #this is the model we re-trained
            tokenizer = tokenizer)

    prediction_score = sentiment_task(x)
    prediction = prediction_score[0]["label"]
    return (prediction)

### this functions alows making a column iwth the score of the sentiment model for the prediciton
def Sentiment_score_predictions_huggin_face(x, sentiment_task = None, re_trained_model = None, 
    tokenizer = "cardiffnlp/twitter-roberta-base-sentiment-latest"):

    if sentiment_task is None:
        sentiment_task = pipeline("sentiment-analysis",
            model = re_trained_model, #this is the model we re-trained
            tokenizer = tokenizer)

    prediction_score = sentiment_task(x)
    score = prediction_score[0]["score"]
    return (score)

## 4.c Predicting the sentiment of all the tweets
Now that we have our trained sentiment analysis model, we wish to use it to label all the tweets we have with sentiment label.

In [13]:
#tweets_table_all["Predicted_sentiment"] = tweets_table_all["text"].apply(lambda x: Sentiment_predictions_huggin_face(x = x, sentiment_task=sentiment_task_model_loaded))
#tweets_table_all["Predicted_sentiment_score"] = tweets_table_all["text"].apply(lambda x: Sentiment_predictions_huggin_face(x = x, sentiment_task=sentiment_task_model_loaded))

In [ ]:
data_folder_name = "data_folder"
#tweets_table_all.to_csv(os.path.join(data_folder_name, "tweets_table_all_with_sentimet.csv"))

## As we have so many tweets, it is very heavy to run the sentiment model on all of them together, so we will perform the labeling task in batches

Creating a directory to store all the mini-csv tables with the sentiment

In [53]:
data_folder_name = "data_folder"
data_folder_name_for_sentiment = os.path.join(data_folder_name, "Predicted_sentiment_tables")
try:
    os.makedirs(data_folder_name_for_sentiment)
    print("Creating directory", data_folder_name_for_sentiment, " to save the sentiment predicted labels")
except:
    print("The dir: ", data_folder_name_for_sentiment, "already exists")
### saving the model we re-trained


Creating directory data_folder\Predicted_sentiment_tables  to save the sentiment predicted labels


In [ ]:
### creating smaller

In [18]:
#tweets_table_all["index_num"] = tweets_table_all.index
#smaller_tweets_table_all = tweets_table_all[["index_num","id", "text"]]
#smaller_tweets_table_all.head()

In [55]:
num_of_rows = tweets_table_all.shape[0]
num_groups = 100
print("Number or rows in each group:", round(num_of_rows/num_groups,1))
split_groups = np.array_split(range(num_of_rows), num_groups)

Number or rows in each group: 9798.2


In [ ]:
# for i in tqdm_notebook(range(len(split_groups))):
#     group_i = split_groups[i]

In [57]:
warnings.filterwarnings('ignore') #ignore warning messages

from tqdm.notebook import tqdm_notebook
name_for_csv_table = "tweets_table_with_sentiment_num"

for i in tqdm_notebook(range(len(split_groups))):
    group_i = split_groups[i]
    sub_group_i = tweets_table_all.iloc[group_i]
    sub_group_i["Predicted_sentiment"] = sub_group_i["text"].apply(lambda x: Sentiment_predictions_huggin_face(x = x, sentiment_task=sentiment_task_model_loaded))
    name_for_csv_table_i = name_for_csv_table + "_" + str(i+1) +".csv"
    sub_group_i.to_csv(os.path.join(data_folder_name_for_sentiment, name_for_csv_table_i))
    
    
warnings.filterwarnings('default') #retrieve warnings

0it [00:00, ?it/s]

C:\Users\roymad\AppData\Local\Temp\ipykernel_39852\1816625370.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_group_i["Predicted_sentiment"] = sub_group_i["text"].apply(lambda x: Sentiment_predictions_huggin_face(x = x, sentiment_task=sentiment_task_model_loaded))
C:\Users\roymad\AppData\Local\Temp\ipykernel_39852\1816625370.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_group_i["Predicted_sentiment"] = sub_group_i["text"].apply(lambda x: Sentiment_predictions_huggin_face(x = x, sentiment_t

C:\Users\roymad\AppData\Local\Temp\ipykernel_39852\1816625370.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_group_i["Predicted_sentiment"] = sub_group_i["text"].apply(lambda x: Sentiment_predictions_huggin_face(x = x, sentiment_task=sentiment_task_model_loaded))
C:\Users\roymad\AppData\Local\Temp\ipykernel_39852\1816625370.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_group_i["Predicted_sentiment"] = sub_group_i["text"].apply(lambda x: Sentiment_predictions_huggin_face(x = x, sentiment_t

C:\Users\roymad\AppData\Local\Temp\ipykernel_39852\1816625370.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_group_i["Predicted_sentiment"] = sub_group_i["text"].apply(lambda x: Sentiment_predictions_huggin_face(x = x, sentiment_task=sentiment_task_model_loaded))
C:\Users\roymad\AppData\Local\Temp\ipykernel_39852\1816625370.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_group_i["Predicted_sentiment"] = sub_group_i["text"].apply(lambda x: Sentiment_predictions_huggin_face(x = x, sentiment_t

C:\Users\roymad\AppData\Local\Temp\ipykernel_39852\1816625370.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_group_i["Predicted_sentiment"] = sub_group_i["text"].apply(lambda x: Sentiment_predictions_huggin_face(x = x, sentiment_task=sentiment_task_model_loaded))
C:\Users\roymad\AppData\Local\Temp\ipykernel_39852\1816625370.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_group_i["Predicted_sentiment"] = sub_group_i["text"].apply(lambda x: Sentiment_predictions_huggin_face(x = x, sentiment_t

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [47]:
sub_group_i

,index_num,id,text,Predicted_sentiment
0,0,1079747729419657217,rt downingstreet more than countries places in...,Neutral
1,1,1079747690110570496,rt gavinwilliamson want to thank all those ser...,Positive
2,2,1078943309744070657,rt ukgovwales welsh secretary aluncairns exten...,Positive
3,3,1078741894761463809,tickettattle tfwrail sure thing,Neutral
4,4,1078038748837888001,wpl_official barrytownunited the_nomads balato...,Positive
...,...,...,...,...
975,975,984543750633508864,rt tradegovuk dit is running summer school for...,Positive
976,976,984417564787953664,rt gw alliance we can be confident that whatev...,Positive
977,977,984392738312421376,rt cancer_wales vale mp aluncairns supports bo...,Neutral
978,978,983700272508755968,rt teamwales tîmcymru teamwales https co za yqhz,Neutral
